2. Build 2 models and unittest them:
    - ready-to-use model from a popular library
    - implement LSTM with PyTorch
    - implement Doc2Vec with NNs (*) - done
    - implement a model using transformers ^^

3. Train & debug models:
    - ready-to-use: 0-small # of bugs expected
    - LSTM: moderate # of bugs expected
    - Transformers: high # of bugs expected

4. Test models

In [28]:
import pandas as pd
df_train = pd.read_csv('Genre Classification Dataset/train_dataframe.csv')
df_train.head()

,title,year,genre,description
0,Oscar et la dame rose,2009,drama,Listening in to a conversation between his doc...
1,Cupid,1997,thriller,A brother and sister with a past incestuous re...
2,"Young, Wild and Wonderful",1980,adult,As the bus empties the students for their fiel...
3,The Secret Sin,1915,drama,To help their unemployed father make ends meet...
4,The Unrecovered,2007,drama,The film's title refers not only to the un-rec...


In [29]:
df_test = pd.read_csv('Genre Classification Dataset/test_dataframe.csv')
df_test.head()

,title,year,genre,description
0,Edgar's Lunch,1998,thriller,"L.R. Brane loves his life - his car, his apart..."
1,La guerra de papá,1977,comedy,"Spain, March 1964: Quico is a very naughty chi..."
2,Off the Beaten Track,2010,documentary,One year in the life of Albin and his family o...
3,Meu Amigo Hindu,2015,drama,"His father has died, he hasn't spoken with his..."
4,Er nu zhai,1955,drama,Before he was known internationally as a marti...


In [30]:
GENRES_NAMES = [
    'action', 'adult', 'adventure',
    'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama',
    'family', 'fantasy', 'game-show', 'history', 'horror', 'music',
    'musical', 'mystery', 'news', 'reality-tv', 'romance', 'sci-fi',
    'short', 'sport', 'talk-show', 'thriller', 'war', 'western'
]
GENRES_NUMBERS = {} 
for i, v in enumerate(GENRES_NAMES):
    GENRES_NUMBERS[v] = i

In [31]:
def join_text(row):
    return '. '.join([row['title'], row['year'], row['description']])
    
df_train['joined_text'] = df_train.apply(join_text, axis=1)
df_test['joined_text'] = df_test.apply(join_text, axis=1)

In [33]:
from tqdm import tqdm
import spacy

nlp = spacy.blank("en")

def make_save_docs(data, outfile):
    """
    this will take a list of texts and labels 
    and transform them in spacy documents
    
    data: list(tuple(text, label))
    
    returns: List(spacy.Doc.doc)
    """
    
    docs = []
    db = spacy.tokens.DocBin()
    
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        
        assert type(doc.text) == str, f"{doc.text} isn't of string type"
        assert type(label), f"{label} isn't of string type"
        
        for GENRE in GENRES_NAMES:
            doc.cats[GENRE] = label == GENRE

        docs.append(doc)
        db.add(doc)

    db.to_disk(outfile)
    
    return docs

datatrain = list(df_train[['joined_text', 'genre']].sample(frac=1).itertuples(index=False, name=None))
datatest = list(df_test[['joined_text', 'genre']].sample(frac=1).itertuples(index=False, name=None))

train_data = make_save_docs(datatrain, 'train.spacy')
dev_data = make_save_docs(datatest, 'dev.spacy')

100%|██████████| 54200/54200 [00:18<00:00, 2994.80it/s]
